# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd07c2b8dc0>
├── 'a' --> tensor([[ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473]])
└── 'x' --> <FastTreeValue 0x7fd07c2b8eb0>
    └── 'c' --> tensor([[ 0.3788,  0.6585, -1.5809,  0.4690],
                        [ 2.2048, -0.2737,  0.9817,  0.4693],
                        [ 0.3709, -0.4852,  1.5215,  1.4743]])

In [4]:
t.a

tensor([[ 0.6715, -0.8230, -0.3787],
        [-2.0332, -0.2442, -1.3473]])

In [5]:
%timeit t.a

59.9 ns ± 0.0671 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd07c2b8dc0>
├── 'a' --> tensor([[ 0.3270, -0.0403,  1.5510],
│                   [-0.2826,  1.3432,  0.6200]])
└── 'x' --> <FastTreeValue 0x7fd07c2b8eb0>
    └── 'c' --> tensor([[ 0.3788,  0.6585, -1.5809,  0.4690],
                        [ 2.2048, -0.2737,  0.9817,  0.4693],
                        [ 0.3709, -0.4852,  1.5215,  1.4743]])

In [7]:
%timeit t.a = new_value

61.7 ns ± 0.0219 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6715, -0.8230, -0.3787],
               [-2.0332, -0.2442, -1.3473]]),
    x: Batch(
           c: tensor([[ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6715, -0.8230, -0.3787],
        [-2.0332, -0.2442, -1.3473]])

In [11]:
%timeit b.a

56 ns ± 0.0438 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0252, -1.0952, -0.8525],
               [-1.0536, -0.8450,  2.2696]]),
    x: Batch(
           c: tensor([[ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.0844 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

822 ns ± 0.152 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 20.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 674 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 496 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcfa7b295b0>
├── 'a' --> tensor([[[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]],
│           
│                   [[ 0.6715, -0.8230, -0.3787],
│                    [-2.0332, -0.2442, -1.3473]]])
└── 'x' --> <FastTreeValue 0x7fd0802ea3d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 74.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd07c262f40>
├── 'a' --> tensor([[ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473],
│                   [ 0.6715, -0.8230, -0.3787],
│                   [-2.0332, -0.2442, -1.3473]])
└── 'x' --> <FastTreeValue 0x7fcfa7b98f10>
    └── 'c' --> tensor([[ 0.3788,  0.6585, -1.5809,  0.4690],
                        [ 2.2048, -0.2737,  0.9817,  0.4693],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 57.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.2 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3788,  0.6585, -1.5809,  0.4690],
                       [ 2.2048, -0.2737,  0.9817,  0.4693],
                       [ 0.3709, -0.4852,  1.5215,  1.4743]],
              
                      [[ 0.3788,  0.6585, -1.5809,  0.4690],
                       [ 2.2048, -0.2737,  0.9817,  0.4693],
                       [ 0.3709, -0.4852,  1.5215,  1.4743]],
              
                      [[ 0.3788,  0.6585, -1.5809,  0.4690],
                       [ 2.2048, -0.2737,  0.9817,  0.4693],
                       [ 0.3709, -0.4852,  1.5215,  1.4743]],
              
                      [[ 0.3788,  0.6585, -1.5809,  0.4690],
                       [ 2.2048, -0.2737,  0.9817,  0.4693],
                       [ 0.3709, -0.4852,  1.5215,  1.4743]],
              
                      [[ 0.3788,  0.6585, -1.5809,  0.4690],
                       [ 2.2048, -0.2737,  0.9817,  0.4693],
                       [ 0.3709, -0.4852,  1.5215,  1.4743]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743],
                      [ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743],
                      [ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743],
                      [ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743],
                      [ 0.3788,  0.6585, -1.5809,  0.4690],
                      [ 2.2048, -0.2737,  0.9817,  0.4693],
                      [ 0.3709, -0.4852,  1.5215,  1.4743],
                      [ 0.3788,  0.6585, -1.5809,  0.4690],
                   

In [28]:
%timeit Batch.cat(batches)

149 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 726 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
